# Speed up loading by converting the dataset from CSV format to Parquet format
- CSV files are ubiquitous, and are often entirely appropriate. 
- Hoever when we start working with large tabular datasets, CSV can become tedious, and its worth considering your options.
- One approach might be to use a faster library than Paandas, such as POLARS or CuDF.
- Another is to convert the CSV into a more efficient format. 
- There are many options for tabular format data, inlcuding parquet, feather, pickle, and more.
- In this instance we'll use Parquet
- Read about parquet files [here](https://databricks.com/glossary/what-is-parquet)

## Excerpt from DataBricks:

**What is Parquet?**

*Parquet is an open source file format available to any project in the Hadoop ecosystem. Apache Parquet is designed for efficient as well as performant flat columnar storage format of data compared to row based files like CSV or TSV files.*

*Parquet uses the record shredding and assembly algorithm which is superior to simple flattening of nested namespaces. Parquet is optimized to work with complex data in bulk and features different ways for efficient data compression and encoding types.  This approach is best especially for those queries that need to read certain columns from a large table. Parquet can only read the needed columns therefore greatly minimizing the IO.*

Lets look at the performance differences below!

In [6]:
import pandas as pd
import numpy as np
import gc
import time

# Reading as CSV (Slow)


In [7]:
%%time
train = pd.read_csv('./../data/train.csv')

CPU times: user 1min 37s, sys: 12.2 s, total: 1min 49s
Wall time: 2min 5s


- the file is **18GB in size**
- it takes 2:20s to load into memory on my computer. Times will vary.
- This is inefficient when done repeatedly

In [ ]:
# convert to parquet

# Reading as Parquet (Fast)
- **5.5GB** in size.
- This is faster and keeps the dtypes of the original dataset.

In [8]:
%%time
train = pd.read_parquet('./../data_parquet/train.parquet')

CPU times: user 24.1 s, sys: 38 s, total: 1min 2s
Wall time: 32.8 s


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 304 entries, row_id to f_299
dtypes: float64(301), int64(2), object(1)
memory usage: 7.1+ GB


In [10]:
train.dtypes

row_id            object
time_id            int64
investment_id      int64
target           float64
f_0              float64
                  ...   
f_295            float64
f_296            float64
f_297            float64
f_298            float64
f_299            float64
Length: 304, dtype: object

In [11]:
del train
gc.collect()

545

# Reading as Parquet Low Memory (Fast & Low Mem Use)
- **3.63GB** in size
- Even better! Uses less memory and loads even faster!

In [14]:
%%time
train = pd.read_parquet('./../data_parquet/train_low_mem.parquet')

CPU times: user 15 s, sys: 18 s, total: 33 s
Wall time: 5.56 s


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 304 entries, row_id to f_299
dtypes: float32(301), object(1), uint16(2)
memory usage: 3.6+ GB


In [ ]:
train.dtypes

# Read just a single `investment_id`
- If you only want to work with a single transaction load them like this

In [16]:
%%time
example = pd.read_parquet('./../data_parquet/investment_ids/529.parquet')

CPU times: user 39.9 ms, sys: 19.3 ms, total: 59.2 ms
Wall time: 178 ms


In [17]:
example.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1194 entries, 315 to 3138440
Columns: 304 entries, row_id to f_299
dtypes: float64(301), int64(2), object(1)
memory usage: 2.8+ MB


# Reading a Subset of Columns

In [18]:
%%time
col_subset = ['time_id','investment_id','target','f_1','f_2','f_3']
train = pd.read_parquet('./../data_parquet/train.parquet', columns=col_subset)

CPU times: user 139 ms, sys: 209 ms, total: 348 ms
Wall time: 335 ms


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time_id        int64  
 1   investment_id  int64  
 2   target         float64
 3   f_1            float64
 4   f_2            float64
 5   f_3            float64
dtypes: float64(4), int64(2)
memory usage: 143.8 MB


# Conclusion
When working with tabular data, IO considerations become an issue with bigger datasets. one way to deal with this is to convert older formats like CSV into more efficient modern formats like parquet, which retain datatypes and offer both smaller file size and faster read write times than CSV.